## Feature Engineering and CV based Winners' Solutions

continued from eda_4_29

New in this notebook

To-do: 
- ~~number of active month~~
- ~~More features related to history products~~
- ~~new product each month in history~~
- ~~number of new products in each month~~
- 


Features:
- eda_4_28
    - average of products for each (customer, product) pair
    - exponent weighted average of products each (customer, product) pair
    - time since presence of products, distance to the first 1
    - time to the last positive flank (01)
    - time to the last negative flank (10)
    - time to the last 1, to the nearest product purchase
    - time to the first 1, to the first product purchase
- before eda_4_25
    - customer info in the second month
    - products in the first month
    - combination of first and second month `ind_actividad_cliente`
    - combination of first and second month `tiprel_1mes`
    - combination of first month product by using binary number (`target_combine`)
    - encoding `target_combine` with 
        - mean number of new products
        - mean number of customers with new products
        - mean number of customers with each new products
    - Count patterns in the last `max_lag` months
    - Number of month to the last time the customer purchase each product
        - CV@2015-12-28: mlogloss=1.29349
        - Private score: 0.0302475, public score: 0.0299266
- eda_4_25
    - Use all available history data
        - E.g., for 2016-05-28 train data, use all previous months, for 2015-02-28, use 1 lag month. 
        - Need to create test set that use the same amount of previous months for each training data set. 
        - This is from [the second winner's solution](https://www.kaggle.com/c/santander-product-recommendation/discussion/26824), his bold part in paragraph 4.
    - Combine models trained on 2016-05-28 and 2015-06-28:
        - Private score: 0.0304583, public score: 0.0300839
        - This is to catch both seasonality and trend, presented in 2015-06-28 and 2016-05-28, respectively. 
        - This idea is mentioned by many winners, like [11-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823) and [14-th winner](https://www.kaggle.com/c/santander-product-recommendation/discussion/26808)

- eda_4_27
    - put 2015-06-28 and 2016-05-28 in the same data set, with the same lag=5
        - Private score:0.0303096, public score: 0.0299867
        - Different as [11-th winner's discussion](https://www.kaggle.com/c/santander-product-recommendation/discussion/26823)
            > We tested this by adding 50% of May-16 data to our June model and sure enough, we went from 0.0301 to 0.0303. Then, we built separate models for Jun and May, but the ensemble didn’t work. We weren’t surprised because June data is better for seasonal products, and May data is better for trend products. And vice-versa, June data is bad for trend products and May data is bad for seasonal products. So, they sort of cancelled each other out.

        - But my score is always worse than theirs, maybe this is the reason why we have different observations

# Draft

In [1]:
from santander_helper import *

In [7]:
x_train, y_train, weight_train = create_train('2015-06-28', max_lag=5, fixed_lag=6, pattern_flag=True)

In [8]:
x_test = create_test(pattern_flag=True)

In [4]:
x_train.head().T.to_csv('x_train_head.csv')

In [5]:
x_test.shape

(929615, 547)

In [6]:
x_test.head().T.to_csv('x_test_head.csv')

In [7]:
x_train.shape

(45140, 547)

In [9]:
customer_product_pair = calculate_customer_product_pair()

In [10]:
month1 = '2015-05-28'
max_lag = 5
fixed_lag = 6
month_new = month_list.index(month1)+1
month_end = month_list.index(month1)
month_start = month_end-max_lag+1
month_start_fixed_lag = month_end-fixed_lag+1

In [11]:
current_month_range = list(range(month_start_fixed_lag, month_new))

In [12]:
current_month_range

[-1, 0, 1, 2, 3, 4]

In [13]:
# New products purchased by each customer in each month
# Count the number of products
n_new_1 = customer_product_pair.groupby(['ncodpers', 'fecha_dato'])['product'].count()
n_new_1 = n_new_1.unstack(level=1).fillna(0.0)
current_month_range = list(range(month_start_fixed_lag, month_new))
n_new = pd.DataFrame(index=n_new_1.index)
for i, m in enumerate(current_month_range):
    if m<1:
        n_new['n_new_products_lag_'+str(fixed_lag-i)] = np.nan
    else:
        n_new['n_new_products_lag_'+str(fixed_lag-i)] = n_new_1[month_list[m]]

In [14]:
customer_product_pair.head()

,ncodpers,product,fecha_dato
0,545113,8,2015-02-28
1,544977,2,2015-02-28
2,544977,11,2015-02-28
3,544977,12,2015-02-28
4,545326,15,2015-02-28


In [15]:
# Target history of each product in each month
history_target = []
for t in range(len(target_cols)):
    tmp = customer_product_pair.loc[customer_product_pair['product']==t, :].copy()
    tmp = tmp.set_index(['ncodpers', 'fecha_dato']).unstack(level=1)
    tmp = tmp.replace({np.nan: 0, t: 1})
    tmp.columns = tmp.columns.levels[1]
    target_tmp = pd.DataFrame(index=tmp.index)
    for i, m in enumerate(current_month_range):
        if m<1:
            target_tmp[target_cols[t]+'_target_lag_'+str(fixed_lag-i)] = np.nan
        else:
            target_tmp[target_cols[t]+'_target_lag_'+str(fixed_lag-i)] = tmp[month_list[m]]
    history_target.append(target_tmp)
history_target = pd.concat(history_target, axis=1)

In [16]:
history_target.head()

,ind_cco_fin_ult1_target_lag_6,ind_cco_fin_ult1_target_lag_5,ind_cco_fin_ult1_target_lag_4,ind_cco_fin_ult1_target_lag_3,ind_cco_fin_ult1_target_lag_2,ind_cco_fin_ult1_target_lag_1,ind_cder_fin_ult1_target_lag_6,ind_cder_fin_ult1_target_lag_5,ind_cder_fin_ult1_target_lag_4,ind_cder_fin_ult1_target_lag_3,...,ind_tjcr_fin_ult1_target_lag_4,ind_tjcr_fin_ult1_target_lag_3,ind_tjcr_fin_ult1_target_lag_2,ind_tjcr_fin_ult1_target_lag_1,ind_valo_fin_ult1_target_lag_6,ind_valo_fin_ult1_target_lag_5,ind_valo_fin_ult1_target_lag_4,ind_valo_fin_ult1_target_lag_3,ind_valo_fin_ult1_target_lag_2,ind_valo_fin_ult1_target_lag_1
ncodpers,,,,,,,,,,,,,,,,,,,,,
15889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
15891,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15892,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15896,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0


In [17]:
tmp

fecha_dato,2015-02-28,2015-03-28,2015-04-28,2015-05-28,2015-06-28,2015-07-28,2015-08-28,2015-09-28,2015-10-28,2015-11-28,2015-12-28,2016-01-28,2016-02-28,2016-03-28,2016-04-28,2016-05-28
ncodpers,,,,,,,,,,,,,,,,
15896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
15913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
16007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16055,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
16287,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [51]:
114/19

6.0

In [ ]:
history = distance_last_one.join((distance_first_one, distance_negative_flank, 
    distance_positive_flank, mean_exp_product, mean_product, 
    distance_positive_flank_first, distance_negative_flank_first, 
    valid_active, lags))

In [47]:
tmp['2015-02-28']

ncodpers
15891      0.0
15892      0.0
15896      0.0
15913      0.0
15921      0.0
15927      0.0
15944      0.0
15950      0.0
15951      0.0
15957      0.0
15965      0.0
15969      0.0
15984      0.0
15994      0.0
15998      0.0
16007      0.0
16019      0.0
16100      0.0
16133      0.0
16138      0.0
16162      1.0
16192      0.0
16196      0.0
16202      0.0
16203      0.0
16208      0.0
16227      0.0
16254      1.0
16291      0.0
16296      0.0
          ... 
1553102    0.0
1553119    0.0
1553120    0.0
1553147    0.0
1553152    0.0
1553180    0.0
1553181    0.0
1553188    0.0
1553218    0.0
1553232    0.0
1553241    0.0
1553253    0.0
1553257    0.0
1553266    0.0
1553273    0.0
1553288    0.0
1553296    0.0
1553303    0.0
1553338    0.0
1553389    0.0
1553390    0.0
1553393    0.0
1553415    0.0
1553429    0.0
1553436    0.0
1553456    0.0
1553541    0.0
1553559    0.0
1553565    0.0
1553571    0.0
Name: 2015-02-28, Length: 149902, dtype: float64

In [22]:
target_history

In [21]:
dn

,n_new_products_lag_1,n_new_products_lag_2,n_new_products_lag_3,n_new_products_lag_4,n_new_products_lag_5
ncodpers,,,,,
15889,0.0,0.0,0.0,1.0,0.0
15891,0.0,0.0,0.0,0.0,0.0
15892,0.0,0.0,0.0,0.0,1.0
15894,0.0,0.0,1.0,0.0,0.0
15896,0.0,0.0,0.0,0.0,1.0
15897,0.0,0.0,1.0,0.0,1.0
15900,0.0,0.0,1.0,0.0,0.0
15901,0.0,0.0,0.0,0.0,0.0
15906,0.0,1.0,1.0,0.0,1.0
